正則表達式:TVBS清理作者格式方法

In [ ]:
import re
#作者欄位清理方法，要放author[0]
def tvbs_author_etl(author_text):
    author = []
    author_text = re.sub(r"記者  ", "", author_text)
    author_text = re.sub(r" / 攝影.*", "", author_text)
    author_text = re.sub(r" / ", "", author_text)
    author_text = re.sub(r"編輯  ", "", author_text)
    author_text = re.sub(r" 報導", "", author_text)
    author_text = re.sub(r" ", ",", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find(",") != -1:
        author_text = author_text.split(",")
        for a in author_text:
            author.append(a)
    elif author_text == "":
        author = []
    else:
        author.append(author_text)
    return author

正則表達式:SETN清理作者格式方法

In [ ]:
#SETN作者欄位清理方法，要放author[0]
def setn_author_etl(author_text):
    author = []
    #將／XX報導XX字串清除
    author_text = re.sub(r"／.*報導.*", "", author_text)
    #將／XX特稿XX字串清除
    author_text = re.sub(r"／.*特稿.*", "", author_text)
    #將記者字串清除
    author_text = re.sub(r"記者", "", author_text)
    #將文／字串清除
    author_text = re.sub(r"文／", "", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find("、") != -1:
        author_text = author_text.split("、")
        for a in author_text:
            author.append(a)
    elif author_text == "":
        author = []
    else:
        author.append(author_text)
    return author

正則表達式:ETtoday清理作者格式方法

In [ ]:
#ETtoday作者欄位清理方法，要放author[0]
def ettoday_author_etl(author_text):
    author = []
    author_text = re.sub("／.?.?報導.*", "", author_text)
    author_text = re.sub(".*記者", "", author_text)
    author_text = re.sub("\xa0", "", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find("、") != -1:
        author_text = author_text.split("、")
        for a in author_text:
            author.append(a)
    elif author_text == "":
        author = []
    else:
        author.append(author_text)
    return author

文章正反面情緒分析 需要的套件

In [ ]:
#!pip install udicOpenData
#!pip install Swinger
#在dockerfile的指令
#ADD ./dict_all.txt /opt/conda/lib/python3.6/site-packages/udicOpenData/dictionary/
from udicOpenData.dictionary import *
jieba.load_userdict(os.path.join(DIR_NAME, 'dict_all.txt'))
#要等一下子
from Swinger import Swinger

文章正反面情緒分析方法

In [ ]:
s = None
def emo_swinger(content, model='LogisticRegression'):
    global s
    if s == None:
        s = Swinger()
        s.load(model)
    # default model= "LogisticRegression", 可替換為"MultinomialNB"
    emo_result = s.swing(content)
    if emo_result == "pos":
        result = "positive"
    elif emo_result == "neg":
        result = "negative"
    return result

文章關鍵字 需要的套件

In [ ]:
import jieba.analyse

文章關鍵字方法

In [ ]:
def keyword_analysis(content):
    #default kewwords = 8
    kws =jieba.analyse.extract_tags(content, 5)
    return kws

文章藍綠貼標分析 需要的套件

In [ ]:
!pip install pandas 
import jieba
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib

將jieba斷詞寫成方法

In [ ]:
def cutflow(p):
    cutresult = " ".join(jieba.cut(p))
    return cutresult.replace("\r", "").replace("\n", "")

載入詞向量模型與貝氏模型

In [ ]:
clg = joblib.load('clg') 
vec = joblib.load('vec') 

文章藍綠貼標分析方法:貝氏定理

In [ ]:
def label_predict(text):
    docs_news = cutflow(text)
    c=[{"content" : docs_news,
       "tag" : 0}]
    d = pd.DataFrame(c)
    test_counts = vec.transform(d["content"])
    pre = clg.predict(test_counts)
    label_result = list(pre)[0]
    if label_result == '1':
        result = "blue"
    elif label_result == '0':
        result = "green"
    return result

文章摘要分析 需要的套件

In [ ]:
!pip install networkx
!pip install textrank4zh
from  __future__  import print_function
import jieba
import numpy
import networkx
import sys
import codecs 
from textrank4zh import TextRank4Keyword ,TextRank4Sentence

文章摘要方法:TextRank

In [ ]:
def news_summary(text):
    tr4s = TextRank4Sentence()
    tr4s.analyze( text = text, lower = True , source = ' all_filters ' )
    for item in tr4s.get_key_sentences( num = 1 ):
        return item.sentence

Elasticsearch 需要的套件

In [ ]:
!pip install elasticsearch
from elasticsearch import Elasticsearch

In [ ]:
# 建立Elasticsearch連線
es = Elasticsearch(['elasticsearch:9200'])

In [ ]:
# 在Elasticsearch建立index，亦可加入body參數先定義mapping格式
es.indices.create(index = 'news', ignore = 400)

In [ ]:
# 將Elasticsearch定義所放入的資料格式:mapping
es.indices.put_mapping(index = "news",
                       doc_type = "politics",                
                       body = {
                        "properties": {
                            "source": {"type": "text"},
                            "url": {"type": "text"},
                            "title": {"type": "text"},
                            "date_": {"type": "text"},
                            "author": {"type": "text"},
                            "content": {"type": "text"},
                            "kw": {"type": "text"},
                            "img_url": {"type": "text"},
                            "@timestamp": {"type": "date"}
                        }
                    }
                )

In [ ]:
# 為了將匯入資料的時間加入時區
from datetime import datetime, timedelta, timezone
tz_utc_8 = timezone(timedelta(hours=8)) # 創建時區UTC+8:00

Kafka Consumer需要的套件

In [ ]:
!pip install kafka
from kafka import KafkaConsumer, TopicPartition
import sys
import json, time, requests

In [ ]:
if __name__ == "__main__":
    # 步驟1.設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
    consumer = KafkaConsumer(
        # Kafka集群在那裡?
        bootstrap_servers=["kafka1:29092"],
        # ConsumerGroup的名稱
        group_id="test01",
        # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
        key_deserializer=bytes.decode,
        # 指定msgValue的反序列化器
        #value_deserializer=bytes.decode,
        value_deserializer=lambda m: json.loads(m.decode('utf-8')),
        # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
        auto_offset_reset="earliest",
    )
    # 步驟2.指定想要訂閱訊息的topic名稱
    topic_name = "test"
    # 步驟3.讓Consumer向Kafka集群訂閱指定的topic
    consumer.subscribe(topics = topic_name)


    # 步驟4.持續的拉取Kafka有進來的訊息
    try:
        print("Start listen incoming messages ...","\n")
        # 持續監控是否有新的record進來 - 方法一
        for record in consumer:
            topic = record.topic
            partition = record.partition
            offset = record.offset
            timestamp = record.timestamp
        
            # 取出msgKey與msgValue
            msgKey = record.key
            msgValue = record.value
            
            # 將新聞作者欄位做處理後再放回
            if msgValue["source"] == "TVBS":      
                msgValue["author"] = tvbs_author_etl(msgValue["author"][0])
            elif msgValue["source"] == "SETN":
                msgValue["author"] = setn_author_etl(msgValue["author"][0])
            elif msgValue["source"] == "ETtoday":
                msgValue["author"] = ettoday_author_etl(msgValue["author"][0])
            
            # 將新聞內文做文章情緒分析並加入欄位與結果
            msgValue["ariticle_emotion"] = emo_swinger(msgValue["content"])
            # 將新聞內文做關鍵字分析再合併至新聞抓到的關鍵字列表，並把重複的篩選掉
            content_keyword = keyword_analysis(msgValue["content"])
            for k in content_keyword:
                msgValue["kw"].append(k)
            msgValue["kw"] = list(set(msgValue["kw"]))
            # 將新聞內文做藍綠分析並加入欄位結果
            msgValue["label"] = label_predict(msgValue["content"])
            # 將新聞做文章摘要並加入欄位與結果
            msgValue["abstract"] = news_summary(msgValue["content"])
            
            # 秀出metadata與msgKey & msgValue訊息
            print("topic=%s, partition=%s, offset=%s : (key=%s, value=%s)" % (record.topic, record.partition,
                                                                              record.offset, record.key, msgValue))
            print("\n")
            
            print("Start sending data to MySQL !!!")
            # 將資料送入MySQL
            ip_location = 'chatbot_api'
                         
            News = msgValue
            
            # 將json傳回API Server
            Endpoint = 'http://%s:5000/news' % (ip_location)

            # header要特別註明是json格式
            Header = {'Content-Type':'application/json'}
            
            # 傳送post對API server新增資料 
            Response = requests.post(Endpoint,headers=Header,data=json.dumps(News))

            # 印出Response的資料訊息
            print(Response)
            Response = Response.json()
            print(Response)
            print("\n")
            
        
            print("Start sending data to elasticsearch !!!")
            # 將資料送入Elasticsearch
            elasticsearch_data = msgValue
            elasticsearch_data["@timestamp"] = datetime.now().replace(tzinfo = tz_utc_8)            
            # 將資料放入Elasticsearch，若沒有index，會直接建立index、type 
            es.index(index = 'news', doc_type = 'politics', body = elasticsearch_data, refresh = True )
            print("\n")

    except:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
    finally:
        consumer.close()